In [1]:
%pip install pandas
%pip install numpy
%pip install spacy
%pip install -U scikit-learn
%pip install seaborn
%pip install transformers
%pip install fasttext
%pip install lazypredict
%pip install gensim

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/e3/59/35a2892bf09ded9c1bf3804461efe772836a5261ef5dfb4e264ce813ff99/pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Obtaining dependency information for numpy>=1.21.0 from https://files.pythonhosted.org/packages/71/3c/3b1981c6a1986adc9ee7db760c0c34ea5b14ac3da9ecfcf1ea2a4ec6c398/numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 7.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/5

In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
import fasttext
import fasttext.util
from transformers import BertTokenizer
import gensim
import gensim.downloader

In [3]:
mfrc = pd.read_csv('datasets/edited/mfrc_sentiment_score.csv')
mfrc

,text,subreddit,bucket,annotator,annotation,confidence,care.virtue,care.vice,fairness.virtue,fairness.vice,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_text,positive,neutral,negative
0,That particular part of the debate is especial...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,particular debate especially funny macron expl...,0.1067,0.3189,0.5743
1,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,r france pretty lively lingo usually delibera...,0.1703,0.5590,0.2707
2,TBH Marion Le Pen would be better. Closet fasc...,neoliberal,French politics,annotator03,Non-Moral,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,tbh marion le pen well closet fascist vs flamb...,0.0658,0.4460,0.4882
3,it really is a very unusual situation isn't it...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,unusual situation fillon affair influence vote...,0.0204,0.3472,0.6324
4,The Le Pen brand of conservatism and classical...,europe,French politics,annotator03,Authority,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,le pen brand conservatism classical right wing...,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,My job is actually ok and I work with loads of...,antiwork,US Politics,annotator00,Non-Moral,Not Confident,0,0,0,0,0,0,0,0,0,0,job actually ok work load nice people decent p...,0.0765,0.1741,0.7494
17882,Someone dying of a disease doesn't change that...,Conservative,US Politics,annotator00,Non-Moral,Not Confident,0,0,0,0,0,0,0,0,0,0,die disease change massively react lethal peop...,0.0100,0.1579,0.8321
17883,Some of these politicians and commentators tak...,politics,US Politics,annotator00,Equality,Not Confident,0,0,0,0,0,0,0,0,0,0,politician commentator take offense biden call...,0.0186,0.2494,0.7321
17884,"No, it's ""truth over facts! Wait where am I ag...",Conservative,US Politics,annotator00,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,truth fact wait biden,0.0470,0.4317,0.5214


In [2]:
davidson = pd.read_csv("datasets/edited/davidson_sentiment_score.csv")
davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,care.virtue,care.vice,fairness.virtue,...,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_tweets,positive,neutral,negative
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,0,...,0,0,0,0,1,1,rt mayasolovely woman complain clean house am...,0.0679,0.4113,0.5208
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,0,0,...,0,0,0,0,0,0,rt mleew boy dat cold tyga dwn bad cuffin d...,0.0326,0.4309,0.5365
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,0,0,...,0,0,0,0,0,2,rt urkindofbrand dawg rt sbaby life fuck b...,0.0031,0.0268,0.9701
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,0,...,0,0,0,0,0,0,rt c g anderson viva based look like tranny,0.0609,0.7310,0.2081
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,0,0,...,0,0,0,0,0,1,rt shenikaroberts shit hear true faker bitch ...,0.0119,0.1853,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,0,0,...,0,0,0,0,1,1,muthaf in lie lifeasking pearls c...,0.0087,0.1202,0.8711
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",0,0,0,...,0,0,0,0,0,0,go break wrong heart baby drive redneck crazy,0.0087,0.1022,0.8891
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,0,0,0,...,0,0,0,0,0,0,young buck wanna eat dat nigguh like be nt fuc...,0.0389,0.3873,0.5738
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,0,0,0,...,0,0,0,0,0,0,youu got wild bitch tellin lie,0.0096,0.1096,0.8808


In [9]:
from pathlib import Path

def getTrainSetFastText():
    ft_model = fasttext.load_model('fasttext_model.bin')
    traindata = []
    for tweet in davidson['cleaned_tweets']:
        traindata.append(ft_model.get_sentence_vector(tweet.replace("\n", "")))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(davidson['cleaned_tweets']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    training_data["positive"] = davidson["positive"].values.tolist()
    training_data["neutral"] = davidson["neutral"].values.tolist()
    training_data["negative"] = davidson["negative"].values.tolist()
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(davidson['cleaned_tweets'].tolist(), padding='max_length', max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

def getTrainSetWord2Vec():
    #w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    w2v_model = gensim.downloader.load('word2vec-google-news-300')
    traindata = []
    for msg in davidson['cleaned_tweets']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in davidson['cleaned_tweets']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)

    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

In [4]:
from pathlib import Path

def getTrainSetFastText():
    ft_model = fasttext.load_model('fasttext_model.bin')
    traindata = []
    for tweet in mfrc['cleaned_text']:
        traindata.append(ft_model.get_sentence_vector(tweet.replace("\n", "")))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = mfrc["positive"].values.tolist()
    traindata["neutral"] = mfrc["neutral"].values.tolist()
    traindata["negative"] = mfrc["negative"].values.tolist()
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(mfrc['cleaned_text']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    training_data["positive"] = mfrc["positive"].values.tolist()
    training_data["neutral"] = mfrc["neutral"].values.tolist()
    training_data["negative"] = mfrc["negative"].values.tolist()
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(mfrc['cleaned_text'].tolist(), padding='max_length', max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = mfrc["positive"].values.tolist()
    traindata["neutral"] = mfrc["neutral"].values.tolist()
    traindata["negative"] = mfrc["negative"].values.tolist()
    return traindata

def getTrainSetWord2Vec():
    #w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    w2v_model = gensim.downloader.load('word2vec-google-news-300')
    traindata = []
    for msg in mfrc['cleaned_text']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = mfrc["positive"].values.tolist()
    traindata["neutral"] = mfrc["neutral"].values.tolist()
    traindata["negative"] = mfrc["negative"].values.tolist()
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in mfrc['cleaned_text']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)

    traindata["positive"] = mfrc["positive"].values.tolist()
    traindata["neutral"] = mfrc["neutral"].values.tolist()
    traindata["negative"] = mfrc["negative"].values.tolist()
    return traindata

In [8]:
X = getTrainSetFastText()
X

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.013513,0.007388,0.005629,0.017995,-0.005638,0.003859,0.008448,0.023875,0.008568,0.014286,...,-0.007953,0.025058,0.007362,0.011026,0.033907,0.030561,-0.014257,0.1067,0.3189,0.5743
1,-0.006746,-0.009040,-0.021371,-0.038347,0.020246,0.003266,-0.014383,-0.028080,-0.011928,-0.027390,...,0.032626,0.039031,0.021194,-0.006173,0.001381,0.012253,-0.000731,0.1703,0.5590,0.2707
2,-0.015231,0.015198,0.004071,-0.007274,-0.052219,0.038719,-0.019969,-0.012492,0.002779,-0.007898,...,-0.022361,-0.046067,0.017683,-0.041864,0.050589,0.014222,-0.025601,0.0658,0.4460,0.4882
3,-0.051756,0.024285,-0.014978,0.031383,0.009304,-0.019355,-0.029879,-0.041641,-0.028755,-0.024432,...,-0.034322,0.049807,-0.045273,0.003333,-0.015226,-0.000679,-0.008870,0.0204,0.3472,0.6324
4,0.008008,-0.007139,0.002471,-0.002554,0.014925,-0.020578,0.006315,-0.006447,-0.004054,-0.033472,...,0.003685,0.038006,0.019404,0.010041,0.006131,-0.011361,0.003301,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.011781,0.021915,0.011811,0.006030,0.016572,-0.014147,-0.009280,0.023295,-0.011605,-0.017095,...,-0.019762,0.015113,0.048323,-0.010257,0.024923,0.017258,-0.003424,0.0765,0.1741,0.7494
17882,0.023371,0.014006,0.024642,0.010196,0.020250,-0.023074,0.002934,-0.056795,0.024968,-0.035613,...,0.003474,0.034382,0.025651,-0.019515,0.028184,0.058080,0.017000,0.0100,0.1579,0.8321
17883,-0.051195,-0.000555,0.035692,-0.036047,0.024744,0.017745,0.021808,0.006061,0.013759,-0.001616,...,0.029434,0.035912,0.018904,0.000107,0.004050,0.018399,-0.002813,0.0186,0.2494,0.7321
17884,-0.004968,0.022924,-0.024582,-0.036626,-0.030045,-0.016713,-0.002322,0.046186,0.002826,0.005633,...,0.082076,0.079354,0.104864,-0.010875,0.050719,-0.025109,0.056572,0.0470,0.4317,0.5214


In [26]:
X.to_csv("fasttext_embedded_davidson.csv")

In [9]:
X.to_csv("datasets/edited/fasttext_embedded_mfrc.csv")

In [10]:
X = getTrainSetBERT()
X

,0,1,2,3,4,5,6,7,8,9,...,238,239,240,241,242,243,244,positive,neutral,negative
0,101,3327,5981,2926,6057,26632,2078,4863,2228,1042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1067,0.3189,0.5743
1,101,1054,2605,3492,17133,17002,2080,2788,9969,2919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1703,0.5590,0.2707
2,101,26419,2232,10115,3393,7279,2092,9346,14870,5443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0658,0.4460,0.4882
3,101,5866,3663,6039,2239,6771,3747,14303,7731,2283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0204,0.3472,0.6324
4,101,3393,7279,4435,9530,8043,20203,6491,4556,2157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,101,3105,2941,7929,2147,7170,3835,2111,11519,3477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0765,0.1741,0.7494
17882,101,3280,4295,2689,5294,2135,10509,12765,2111,4452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0100,0.1579,0.8321
17883,101,3761,12268,2202,10048,7226,2368,2655,2317,22006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0186,0.2494,0.7321
17884,101,3606,2755,3524,7226,2368,102,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0470,0.4317,0.5214


In [30]:
X.to_csv("BERT_embedded_davidson.csv")

In [11]:
X.to_csv("datasets/edited/BERT_embedded_mfrc.csv")

In [5]:
X = getTrainSetGlove()
X

[==================================================] 100.0% 128.1/128.1MB downloaded


,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,-0.210833,0.272271,0.400721,-0.209351,-0.160809,0.217836,-0.139331,-0.183866,0.200468,-0.043891,...,0.175768,-0.192087,-0.139639,-0.295082,-0.372213,0.234299,0.176420,0.1067,0.3189,0.5743
1,-0.274073,0.162306,0.227609,-0.311724,-0.091764,0.324762,0.001157,0.045032,0.057923,0.013589,...,0.137083,-0.117872,0.066304,-0.128310,-0.333411,0.253320,0.167224,0.1703,0.5590,0.2707
2,0.173693,0.291632,-0.040465,-0.313179,-0.118645,0.592572,0.215612,-0.076660,-0.306570,0.183808,...,0.601510,0.002059,-0.296788,-0.540497,-0.580521,-0.033936,-0.039999,0.0658,0.4460,0.4882
3,-0.261215,0.046749,0.243438,-0.013285,0.018544,0.192758,0.017411,-0.253015,0.216240,0.195082,...,0.312093,-0.240254,0.132613,-0.226719,-0.294348,0.278761,0.175862,0.0204,0.3472,0.6324
4,-0.113398,0.243117,0.060554,-0.036737,0.055164,0.198604,-0.078393,-0.307142,-0.124644,0.132557,...,0.285226,-0.282609,-0.069323,-0.245338,-0.467496,0.335180,0.110977,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.074156,0.383546,0.313101,-0.275059,-0.322074,-0.001317,-0.199212,0.228315,0.107322,-0.049445,...,-0.290595,-0.476025,-0.014907,-0.024284,-0.171817,0.377309,0.230237,0.0765,0.1741,0.7494
17882,-0.020872,0.477383,0.128721,-0.173512,-0.531735,0.338086,-0.259254,-0.017950,0.381546,-0.092971,...,0.151908,-0.325027,0.138792,0.112646,-0.254714,0.398557,-0.297553,0.0100,0.1579,0.8321
17883,-0.273321,0.101413,0.271252,-0.481793,-0.219246,0.369080,0.163586,-0.189313,-0.355565,0.093524,...,0.110051,-0.086643,-0.136826,-0.176865,0.007890,0.151259,0.128401,0.0186,0.2494,0.7321
17884,-0.016740,0.285383,0.585497,-0.244007,-0.440218,-0.030287,-0.203567,0.251612,0.233515,-0.434974,...,0.296435,-0.199795,-0.158613,0.026425,-0.010632,0.035825,0.542787,0.0470,0.4317,0.5214


In [32]:
X.to_csv("glove_embedded_davidson.csv")

In [6]:
X.to_csv("datasets/edited/glove_embedded_mfrc.csv")

In [14]:
X = getTrainSetTFIDF()
X

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1067,0.3189,0.5743
1,0.000000,0.0,0.293464,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1703,0.5590,0.2707
2,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0658,0.4460,0.4882
3,0.000000,0.0,0.000000,0.0,0.00000,0.509745,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0204,0.3472,0.6324
4,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.255489,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.469668,0.0,0.0,0.0,0.0,0.0,0.0765,0.1741,0.7494
17882,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.854115,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0100,0.1579,0.8321
17883,0.000000,0.0,0.000000,0.0,0.56753,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0186,0.2494,0.7321
17884,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0470,0.4317,0.5214


In [7]:
X.to_csv("datasets/edited/tfdif_embedded_davidson.csv")

In [15]:
X.to_csv("datasets/edited/tfdif_embedded_mfrc.csv")

In [16]:
X = getTrainSetWord2Vec()
X

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.065732,0.008404,-0.016626,0.123773,-0.139244,0.025628,0.082030,-0.011632,0.097386,0.029644,...,-0.001558,0.058648,-0.077735,0.113413,0.062751,0.036550,-0.016274,0.1067,0.3189,0.5743
1,0.032118,0.049661,-0.028083,0.142175,-0.078153,0.039569,0.001361,-0.039524,0.052426,0.018910,...,-0.076190,0.001072,-0.043476,0.028205,-0.009829,0.070337,-0.044604,0.1703,0.5590,0.2707
2,0.039848,0.025352,0.062933,0.087025,-0.016108,0.119485,-0.036442,-0.056951,0.160955,-0.006214,...,-0.065030,-0.059896,-0.028900,0.078280,-0.034085,-0.022306,-0.098544,0.0658,0.4460,0.4882
3,0.127966,0.026221,-0.094421,0.048181,-0.122888,0.006323,0.017383,-0.063620,0.044873,-0.009985,...,0.067969,-0.019043,-0.061456,0.088171,0.053143,0.013455,0.002094,0.0204,0.3472,0.6324
4,0.074550,0.001446,0.020623,0.157813,-0.067213,-0.012880,0.043339,-0.046779,0.119955,-0.001163,...,-0.055074,-0.038040,-0.055335,0.043462,-0.056761,0.003919,-0.072790,0.0455,0.7794,0.1751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.070877,0.062979,-0.023935,0.086564,-0.033150,0.026986,0.106635,-0.038784,0.076858,0.055965,...,-0.090957,-0.022169,-0.118010,0.023192,0.089972,0.039437,-0.067402,0.0765,0.1741,0.7494
17882,0.117096,0.059406,0.024857,0.165100,-0.199562,0.070049,0.047401,-0.080551,0.126537,0.057587,...,-0.066360,0.045212,-0.070694,0.135502,-0.075470,0.106796,-0.029682,0.0100,0.1579,0.8321
17883,0.074839,0.002553,0.114802,0.026911,0.011454,-0.054815,-0.018656,-0.049601,0.094564,0.028854,...,-0.064392,-0.011454,-0.020264,0.055461,0.075338,0.101016,-0.001862,0.0186,0.2494,0.7321
17884,0.122559,0.066284,0.092041,0.080261,-0.043648,0.058838,0.099731,0.016937,0.208679,0.108841,...,0.021004,-0.140991,0.012207,0.037781,0.074463,0.152466,-0.147568,0.0470,0.4317,0.5214


In [ ]:
X.to_csv("datasets/edited/word2vec_embedded_davidson.csv")

In [17]:
X.to_csv("datasets/edited/word2vec_embedded_mfrc.csv")